In [1]:
import warnings
warnings.filterwarnings('ignore')
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd

In [88]:
df = pd.read_csv('./datadown/stop.csv', header = None)
df[2] = pd.to_datetime(df[2])
df = df[df[5] != 0]
df.sort_values([0, 2], inplace=True)
# filter data with more than 5 days trip,
df = df[df.groupby([0,7])[1].transform('count') >= 5 ]
df['date_gap'] = df.groupby([0, 7], as_index=False)[2].transform("diff")
df['date_gap'] = df['date_gap'].dt.days.fillna(1)
df.reset_index(inplace=True)

# Function to filter groups based on the condition
def filter_group(group):
    # Check if any row in the group has "flag" > 1
    if (group['date_gap'] > 1).any():
        # If yes, drop all rows in the group
        return pd.DataFrame(columns=group.columns)
    else:
        # If no, keep all rows in the group
        return group

# Apply the filter function to each group based on "key"
filtered_df = df.groupby([0, 7]).apply(filter_group).reset_index(drop=True)


In [96]:
# filtered_df[filtered_df[0] == 7609726]

In [204]:
# import os
# os.makedirs('./dataup/')
# filctered_df.to_csv('./dataup/cleaned_df.csv', index = None)

In [166]:
df = pd.read_csv('./dataup/cleaned_df.csv')
df = df.groupby(['0','7'], as_index=False). head(5).sort_values('0')#.head(50)

In [167]:
df.drop(['index', 'date_gap'], axis = 1, inplace = True)
df['month'] = df['2'].str.slice(5,7)
df['year'] = df['2'].str.slice(0,4)
df['steps'] = df.groupby(['0','7'], as_index=False).cumcount() + 1
df_p = df.pivot_table(values=["3", "4", "5"], index=['0', '1', '7',], columns = ['steps'] )
df_p .columns = df_p.columns.get_level_values(1)
df_p = df_p.reset_index()

In [168]:
df_p.columns = [
    'imo',
    'class',
    'des',
    'lat1',
    'lat2',
    'lat3',
    'lat4',
    'lat5',
    'lon1',
    'lon2',
    'lon3',
    'lon4',
    'lon5',
    'speed1',
    'speed2',
    'speed3',
    'speed4',
    'speed5',
    ]

In [180]:
df_p['imo'] = df_p['imo'].astype(str)
df_p['class'] = df_p['class'].astype(str)
df_p['des'] = df_p['des'].astype(str)

In [181]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12292 entries, 0 to 12291
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   imo     12292 non-null  object 
 1   class   12292 non-null  object 
 2   des     12292 non-null  object 
 3   lat1    12292 non-null  float64
 4   lat2    12292 non-null  float64
 5   lat3    12292 non-null  float64
 6   lat4    12292 non-null  float64
 7   lat5    12292 non-null  float64
 8   lon1    12292 non-null  float64
 9   lon2    12292 non-null  float64
 10  lon3    12292 non-null  float64
 11  lon4    12292 non-null  float64
 12  lon5    12292 non-null  float64
 13  speed1  12292 non-null  float64
 14  speed2  12292 non-null  float64
 15  speed3  12292 non-null  float64
 16  speed4  12292 non-null  float64
 17  speed5  12292 non-null  float64
dtypes: float64(15), object(3)
memory usage: 1.7+ MB


In [196]:
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [201]:
class dataset_loading:
    def __init__(self, dataframe):
        self.features = dataframe.drop(columns=['des'])
        self.target = dataframe['des']
        self.cat_columns = ['imo', 'class']  # List of categorical column names
        self.label_encoders = {col: LabelEncoder() for col in self.cat_columns}
        for col in self.cat_columns:
            self.features[col] = self.label_encoders[col].fit_transform(self.features[col])

        # Use OneHotEncoder for one-hot encoding of categorical features
        self.onehot_encoder = OneHotEncoder(sparse=False)
        self.features_encoded = self.onehot_encoder.fit_transform(self.features[self.cat_columns])

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        # Extract features and target for the given index
        sample = {
            'numerical_features': torch.tensor(self.features.iloc[idx, :2].values, dtype=torch.float32),
            'categorical_features': torch.tensor(self.features.loc[idx, self.cat_columns], dtype=torch.long),
            'target': torch.tensor(self.target[idx], dtype=torch.long)
        }
        return sample

# Split the dataset into training and test sets
train_df, test1_df = train_test_split(df_p, test_size=0.2, random_state=42)

# Create an instance of the custom dataset
dataset_train = dataset_loading(train_df)
dataset_test1 = dataset_loading(test1_df)



In [202]:
# Create DataLoaders for training and test sets
train_dataloader = DataLoader(dataset_train, batch_size=2, shuffle=False)
test1_dataloader = DataLoader(dataset_test1, batch_size=2, shuffle=False)


In [203]:

# Example of iterating through the training DataLoader
for batch, a in train_dataloader:
    print(batch)
    print(a)

KeyError: 0

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

# Define the neural network
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define the training function
def train(model, dataloader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(dataloader)}")

# Define the evaluation function
def evaluate(model, dataloader):
    model.eval()
    all_predictions = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            all_predictions.extend(predictions.numpy())
            all_targets.extend(targets.numpy())
    
    accuracy = accuracy_score(all_targets, all_predictions)
    print(f"Accuracy: {accuracy * 100:.2f}%")

# Hyperparameters
input_size = len(train_dataset[0][0])
hidden_size = 16
output_size = len(df['target'].unique())  # Assuming 'target' is the categorical variable
learning_rate = 0.001
epochs = 20

# Create instances of the neural network, loss function, and optimizer
model = SimpleNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create DataLoaders for training and test sets
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

# Train the model
train(model, train_dataloader, criterion, optimizer, epochs=epochs)

# Evaluate the model on the test set
evaluate(model, test_dataloader)
